In [1]:
import h5py
import torch
import numpy as np
import diffusion_pde as dpde
import matplotlib.pyplot as plt

In [2]:
data_dir = dpde.utils.get_repo_root() / "data"


In [3]:
data_path = data_dir / "llg_test.hdf5"
data_path.exists()

True

# Create Heat dataset with diffusivity but fixed T:

Steps:
1. load heat timeseries data
2. pick a time T
3. create a ```heat_no_time.hdf5``` dataset

In [4]:
with h5py.File(data_path, "r") as f:
    for key in f.keys():
        print(f"{key}: shape {f[key].shape}, dtype {f[key].dtype}")

A: shape (200, 3, 64, 16), dtype float32
U: shape (200, 3, 64, 16, 150), dtype float32
labels: shape (200, 3), dtype float32
t_steps: shape (150,), dtype float64


In [5]:
with h5py.File(data_path, "r") as f:
    data = f["U"][:]
    t_steps = f["t_steps"][:]
    labels = f["labels"][:]
    attrs = dict(f.attrs)

In [6]:
new_data = data[..., ::4]
new_t_steps = t_steps[::4]

In [7]:
for k, v in attrs.items():
    print(f"{k}: {v}")

N: 200
T: 2.235e-09
dx: 7.936507936507936e-09
dy: 8.333333333333334e-09
grid_size: [5.00e-07 1.25e-07 3.00e-09]
h_ext_angle: [  0 360]
h_ext_norm: [ 0 50]
num_t_steps: 150
res: [64 16  1]
seed: 42
t_per_step: 1.5e-11


In [8]:
new_attrs = attrs.copy()
new_attrs["description"] = f"LLG equation test dataset with initial state and final state only."
del new_attrs["t_per_step"], new_attrs["num_t_steps"]

for k, v in new_attrs.items():
    print(f"{k}: {v}")

N: 200
T: 2.235e-09
dx: 7.936507936507936e-09
dy: 8.333333333333334e-09
grid_size: [5.00e-07 1.25e-07 3.00e-09]
h_ext_angle: [  0 360]
h_ext_norm: [ 0 50]
res: [64 16  1]
seed: 42
description: LLG equation test dataset with initial state and final state only.


In [10]:
new_data.shape, new_t_steps.shape

((200, 3, 64, 16, 38), (38,))

In [11]:
new_dataset_name = f"llg_test2.hdf5"
new_data_path = data_dir / new_dataset_name
with h5py.File(new_data_path, "w") as f:
    f.create_dataset("U", data=new_data)
    f.create_dataset("t_steps", data=new_t_steps)
    f.create_dataset("labels", data=labels)
    for key, value in new_attrs.items():
        f.attrs[key] = value